<div style="color:white;
           display:fill;
           border-radius:20px;
           background-color:#F5DEB3;
           font-size:180%;
           font-family:Helvetica, Sans-Serif;
           letter-spacing:0.5px">
<p style="padding: 10px;
          text-align: center;
          font-size:150%;
          color:blue;">
           🏦💵Which Debts Are Worth the Bank's Effort?🏧💱
            
</p>
<style>
        h1{text-align: center;}
 </style>  
    
</div>

<p style="font-family:cursive; font-size:25px; color:'darkcyan';">I brewed this notebook from scratch, If this notebook helped, please consider upvoting and cite me if sharing on other platforms.</p>


<p style="width: 700px;padding: 20px;background: papayawhip;border-radius:10px;font-size:23px;font-size:23px;border-radius:20px">
    <a href="https://www.linkedin.com/in/tusharaggarwalinseec/" target="_blank">Lets connect on LinkedIn!</a>
    
   </p>
<p style="width: 700px;padding: 20px;background: papayawhip;border-radius:10px;font-size:23px;border-radius:20px">
<a href="https://github.com/tushar2704" target="_blank">Follow me on Github too!</a> </p>

<p style="width: 700px;padding: 20px;background: papayawhip;border-radius:10px;font-size:23px;border-radius:20px">
    <a href="https://medium.com/@tushar_aggarwal" target="_blank">Also checkout my Medium posts!</a>
    
   </p>


<div class="alert alert-info" role="alert" style="font-size:17px;border-radius:20px">
    <b>Regression discontinuity: banking recovery</b>
    
</div>
<div class="alert alert-info" role="alert" style="font-size:15px;border-radius:20px">
    <b>Some things to note:</b>
    <br><br>After a debt has been legally declared "uncollectable" by a bank, the account is considered to be "charged-off." But that doesn't mean the bank simply walks away from the debt. They still want to collect some of the money they are owed.
    <br><br>
    In this project, we'll look at a situation where a bank assigned delinquent customers to different recovery strategies based on the expected amount the bank believed it would recover from the customer. The goal for us is to determine in this non-random assignment whether the incremental amount the bank earns exceeded the additional cost of assigning customers to a higher recovery strategy.
    <br></br>
    Regression discontinuity is an intuitive and useful analysis method in any situation of a threshold assignment.
</div>

<p style="width: 700px;padding: 20px;background: papayawhip;border-radius:10px;font-size:17px;border-radius:20px">

 

    
   </p>


In [1]:
import numpy as np
import pandas as pd

In [2]:
bank_data = pd.read_csv("/kaggle/input/bank-recovery-data/bank_recovery_data.csv")

In [3]:
bank_data.head()

,id,expected_recovery_amount,actual_recovery_amount,recovery_strategy,age,sex
0,2030,194,263.540,Level 0 Recovery,19,Male
1,1150,486,416.090,Level 0 Recovery,25,Female
2,380,527,429.350,Level 0 Recovery,27,Male
3,1838,536,296.990,Level 0 Recovery,25,Male
4,1995,541,346.385,Level 0 Recovery,34,Male


In [4]:
bank_data.isnull().sum().sort_values()

id                          0
expected_recovery_amount    0
actual_recovery_amount      0
recovery_strategy           0
age                         0
sex                         0
dtype: int64

In [5]:
bank_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1882 entries, 0 to 1881
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        1882 non-null   int64  
 1   expected_recovery_amount  1882 non-null   int64  
 2   actual_recovery_amount    1882 non-null   float64
 3   recovery_strategy         1882 non-null   object 
 4   age                       1882 non-null   int64  
 5   sex                       1882 non-null   object 
dtypes: float64(1), int64(3), object(2)
memory usage: 88.3+ KB


In [6]:
bank_data.describe(include='all')

,id,expected_recovery_amount,actual_recovery_amount,recovery_strategy,age,sex
count,1882.000000,1882.000000,1882.000000,1882,1882.000000,1882
unique,NaN,NaN,NaN,5,NaN,2
top,NaN,NaN,NaN,Level 1 Recovery,NaN,Male
freq,NaN,NaN,NaN,670,NaN,973
mean,1035.596174,2759.967588,4000.967837,NaN,39.650372,NaN
std,591.458429,2019.826565,4576.506350,NaN,15.453072,NaN
min,6.000000,194.000000,200.425000,NaN,18.000000,NaN
25%,534.250000,1261.250000,1045.190300,NaN,28.000000,NaN
50%,1029.500000,2062.000000,2115.446373,NaN,35.000000,NaN
75%,1551.750000,3569.500000,5417.237207,NaN,50.000000,NaN
